# London Passenger Mode Choice

In [10]:
import os, sys
sys.path.insert(0, os.path.abspath(".."))

#import packages
import pandas as pd
from pycmtensor.dataset import Dataset
from pycmtensor.expressions import Beta, RandomDraws
from pycmtensor.models import MNL

lpmc = pd.read_csv("../data/lpmc.dat", sep="\t")
lpmc = lpmc[lpmc["travel_year"]==2015]
ds = Dataset(df=lpmc, choice="travel_mode")
ds.split(0.8)

# Beta parameters
asc_walk = Beta("asc_walk", 0.0, None, None, 1)
asc_cycle = Beta("asc_cycle", 0.0, None, None, 0)
asc_pt = Beta("asc_pt", 0.0, None, None, 0)
asc_drive = Beta("asc_drive", 0.0, None, None, 0)
b_cost = Beta("b_cost", 0.0, None, None, 0)
b_time = Beta("b_time", 0.0, None, None, 0)
s_time = Beta("s_time", 0.5, None, None, 0)
b_purpose = Beta("b_purpose", 0.0, None, None, 0)
b_licence = Beta("b_licence", 0.0, None, None, 0)
b_car_own = Beta("b_car_own", 0.0, None, None, 0)

rnd_time = RandomDraws('rnd_time', 'lognormal', 100)
U_walk = asc_walk + b_time * ds["dur_walking"] # + rnd_time(s_time) * db["dur_walking"]
U_cycle = asc_cycle + b_time * ds["dur_cycling"] 
U_pt = asc_pt + b_time * (ds["dur_pt_rail"] + ds["dur_pt_bus"] + ds["dur_pt_int"]) + b_cost * ds["cost_transit"]
U_drive = asc_drive + b_time * ds["dur_driving"] + b_licence * ds["driving_license"] + b_cost * (ds["cost_driving_fuel"] + ds["cost_driving_ccharge"])

U = [U_walk, U_cycle, U_pt, U_drive]
mymodel = MNL(ds, locals(), U)

12:48:25 [INFO] n_train_samples:3986 n_valid_samples:997
12:48:25 [WARNING] s_time not in any utility functions
12:48:25 [WARNING] b_purpose not in any utility functions
12:48:25 [WARNING] b_car_own not in any utility functions
12:48:25 [INFO] inputs in MNL: [driving_license, dur_walking, dur_cycling, dur_pt_rail, dur_pt_bus, dur_pt_int, dur_driving, cost_transit, cost_driving_fuel, cost_driving_ccharge]
12:48:41 [INFO] Build time = 00:00:15


In [11]:
from pycmtensor.models import train
from pycmtensor.optimizers import Adam, BFGS, RMSProp
from pycmtensor.scheduler import StepLR, ConstantLR

mymodel.reset_values()
train(mymodel, ds, optimizer=Adam, batch_size=0, base_learning_rate=1., convergence_threshold=0.0001, max_steps=2000, lr_scheduler=ConstantLR, lr_stepLR_drop_every=20, lr_stepLR_factor=0.99, BFGS_warmup=10)

12:48:42 [INFO] Start (n=3986, Step=0, LL=-5525.77, Error=80.34%)
12:48:42 [INFO] Train (Step=0, LL=-9008.61, Error=80.34%, gnorm=2.44949e+00, 0/2000)
12:48:42 [INFO] Train (Step=16, LL=-3798.26, Error=39.12%, gnorm=4.46640e-01, 16/2000)
12:48:43 [INFO] Train (Step=54, LL=-3487.97, Error=35.21%, gnorm=6.80979e-02, 54/2000)
12:48:43 [INFO] Train (Step=87, LL=-3471.01, Error=35.01%, gnorm=9.93509e-03, 87/2000)
12:48:43 [INFO] Train (Step=130, LL=-3470.29, Error=34.70%, gnorm=1.92617e-03, 130/2000)
12:48:43 [INFO] Train (Step=168, LL=-3470.28, Error=34.70%, gnorm=3.22536e-04, 168/2000)
12:48:43 [INFO] Train (Step=189, LL=-3470.28, Error=34.70%, gnorm=8.74120e-05, 189/2000)
12:48:43 [INFO] Model converged (t=0.734)
12:48:46 [INFO] Best results obtained at Step 185: LL=-3470.28, Error=34.70%, gnorm=2.16078e-04


In [6]:
pd.options.display.width = 99
print(mymodel.results.beta_statistics())
print(mymodel.results.model_statistics())
print(mymodel.results.benchmark())

              value   std err     t-test   p-value rob. std err rob. t-test rob. p-value
asc_cycle -2.555589  0.122056 -20.937894       0.0      0.13551  -18.859044          0.0
asc_drive -0.065714  0.075207  -0.873767  0.382245     0.083918   -0.783073     0.433584
asc_pt     0.462504  0.059597   7.760475       0.0     0.067708     6.83084          0.0
asc_walk        0.0         -          -         -            -           -            -
b_cost    -0.137062  0.010515 -13.034681       0.0     0.009242  -14.829647          0.0
b_licence  1.305436  0.072837  17.922822       0.0     0.074698   17.476087          0.0
b_time     1.490019  0.136327  10.929758       0.0      0.12737   11.698364          0.0
                                         value
Number of training samples used         3986.0
Number of validation samples used        997.0
Null. log likelihood              -5525.769323
Final log likelihood              -4206.316416
Accuracy                                53.36%
Likeli

In [4]:
mymodel.results.model_robust_correlation_matrix()

,asc_cycle,asc_pt,asc_drive,b_cost,b_time,b_licence
asc_cycle,1.000000,0.555348,0.446142,0.275918,-0.664268,-0.056280
asc_pt,0.555348,1.000000,0.664742,0.028757,-0.747226,-0.023953
asc_drive,0.446142,0.664742,1.000000,0.001864,-0.612640,-0.621469
b_cost,0.275918,0.028757,0.001864,1.000000,-0.421506,-0.113394
b_time,-0.664268,-0.747226,-0.612640,-0.421506,1.000000,0.114312
b_licence,-0.056280,-0.023953,-0.621469,-0.113394,0.114312,1.000000


In [5]:
# predictions
import numpy as np

print(mymodel.predict(ds, return_choices=False))
print(np.unique(mymodel.predict(ds), return_counts=True))

AttributeError: 'MNL' object has no attribute 'predict'